In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import os
import numpy as np
import tensorflow as tf
import cv2

# Configurações
IMG_SIZE = 128

def preprocess_image(image_array):
    """Pré-processa a imagem para entrada no modelo."""
    if len(image_array.shape) == 3 and image_array.shape[2] == 3:
        image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    image_array = image_array.astype(np.uint8)
    image_array = cv2.equalizeHist(image_array)
    image_array = cv2.GaussianBlur(image_array, (5, 5), 0)
    image_array = image_array.astype(np.float32) / 255.0
    image_array = cv2.resize(image_array, (IMG_SIZE, IMG_SIZE))
    image_array = np.expand_dims(image_array, axis=-1)
    image_array = np.expand_dims(image_array, axis=0)

    return image_array

def image_predict(model_path, preprocessed_image_array):
    """Carrega o modelo salvo e realiza predição na imagem pré-processada."""
    modelo = tf.keras.models.load_model(model_path)
    print("✅ Modelo carregado com sucesso.")
    predicao = modelo.predict(preprocessed_image_array)
    return predicao

def main():
    # 🔽 AJUSTE OS CAMINHOS AQUI:
    path_imagem_teste = '/content/drive/MyDrive/teste_imagem/teste-raioxenf.jpg'   # Exemplo: 'testes/raiox.png'
    path_modelo = '/content/drive/MyDrive/modelo_enfisema.keras'         # Exemplo: 'modelos/enfisema_model.keras'

    # Verificar existência dos arquivos
    if not os.path.exists(path_imagem_teste):
        print(f"❌ Erro: Imagem de teste não encontrada em '{path_imagem_teste}'")
        return

    if not os.path.exists(path_modelo):
        print(f"❌ Erro: Modelo não encontrado em '{path_modelo}'")
        return

    # Carregar imagem
    imagem_original = cv2.imread(path_imagem_teste, cv2.IMREAD_GRAYSCALE)
    if imagem_original is None:
        print(f"❌ Erro ao carregar a imagem: {path_imagem_teste}")
        return

    # Pré-processar imagem
    imagem_preprocessada = preprocess_image(imagem_original)
    print(f"📊 Shape da imagem após pré-processamento: {imagem_preprocessada.shape}")

    # Fazer predição
    predicao_imagem = image_predict(path_modelo, imagem_preprocessada)
    if predicao_imagem is not None:
        print("\n📈 Predição (valor bruto):", predicao_imagem)
        if predicao_imagem[0][0] >= 0.5:
            print("🔴 Resultado: Enfisema detectado (Classe 1)")
        else:
            print("🟢 Resultado: Normal (sem Enfisema) (Classe 0)")

if __name__ == '__main__':
    main()


📊 Shape da imagem após pré-processamento: (1, 128, 128, 1)
✅ Modelo carregado com sucesso.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step

📈 Predição (valor bruto): [[0.968341]]
🔴 Resultado: Enfisema detectado (Classe 1)
